In [2]:
'''
 Distributions by ages and genders:
    - Incubation period                         (incubation)
    - Infectious period                         (infectious)
    - Home to recovery                          (home_to_rec)
    + Illness onset to hospitalisation          (ill_to_hosp)
    + Hospitalisation to critical               (hosp_to_cri)
    + Hospitalisation to death                  (hosp_to_dea)
    + Hospitalisation to recovery -severe       (hosp_to_r_s)
    + Hospitalisation to recovery -critical     (hosp_to_r_c)
'''

from scipy.stats import *
from scipy.optimize import minimize

x0 = [0.5,2.,1]

def generate_lms_function(distribution, mean=None, median=None, std=None,
                          p2h=None, p5=None, p95=None, p97h=None, p99=None):
    
    def function(x):
        funcs = []
        if mean is not None:
            funcs.append(lambda *x: (distribution.mean(*x) - mean)**2)
        if median is not None:
            funcs.append(lambda *x: (distribution.median(*x) - median)**2)
        if std is not None:
            funcs.append(lambda *x: (distribution.std(*x) - std)**2)
        if p2h is not None:
            funcs.append(lambda *x: (distribution.cdf(p2h,*x) - 0.025)**2)
        if p5 is not None:
            funcs.append(lambda *x: (distribution.cdf(p5,*x) - 0.05)**2)
        if p95 is not None:
            funcs.append(lambda *x: (distribution.cdf(p95,*x) - 0.95)**2)
        if p97h is not None:
            funcs.append(lambda *x: (distribution.cdf(p97h,*x) - 0.975)**2)
        if p99 is not None:
            funcs.append(lambda *x: (distribution.cdf(p99,*x) - 0.99)**2)
        return sum(f(*x) for f in funcs)
    
    return function

In [2]:
# data: https://www.thelancet.com/action/showPdf?pii=S1473-3099%2820%2930198-5
#       hospitalised but moderate symptoms
#       no severe/critical/death cases reported
print('Age group: 0-16')
incubation  = None
infectious  = None
home_to_rec = minimize(generate_lms_function(lognorm, mean=14., std=3., p5=10., p95=20.),x0).x
ill_to_hosp = None
hosp_to_cri = None
hosp_to_dea = None
hosp_to_r_s = None
hosp_to_r_c = None
print(incubation,infectious,home_to_rec,ill_to_hosp,hosp_to_cri,hosp_to_dea,hosp_to_r_s,hosp_to_r_c)

Age group: 0-16
(None, None, array([0.32935674, 5.13604047, 8.39607137]), None, None, None, None, None)


In [67]:
# data: https://www.thelancet.com/pdfs/journals/laninf/PIIS1473-3099(20)30243-7.pdf
#       mean, p2, p98

import numpy as np
import matplotlib.pyplot as plt

data_deceased = np.array([(0.00161,0.000185,0.0249),
                          (0.00695,0.00149,0.0502),
                          (0.0309,0.0138,0.0923),
                          (0.0844,0.0408,0.185),
                          (0.161,0.0764,0.323),
                          (0.595,0.344,1.28),
                          (1.93,1.11,3.89),
                          (4.28,2.45,8.44),
                          (7.80,3.80,13.3),])
data_severe = np.array([(0.00,0.00,0.00),
                       (0.0408,0.0243,0.0832),
                       (1.04,0.622,2.13),
                       (3.43,2.04,7.00),
                       (4.25,2.53,8.68),
                       (8.16,4.86,16.7),
                       (11.8,7.01,24.0),
                       (16.6,9.87,33.8),
                       (18.4,11.0,37.6)])*1
x = np.linspace(-1,20,1000)
plt.figure()
# plt.plot(x, lognorm.pdf(x, 1,0,1))
print 'deceased'
x0=[0.5,0.2,0.1]
x0=[0.1,2]
for mean, p2h, p97h in data_deceased:
    res = minimize(generate_lms_function(norm, mean=mean, p2h=p2h, p97h=p97h),x0).x
    print res
    # print norm.pdf(x, *res)
print 'severe'
for mean, p2h, p97h in data_severe:
    res = minimize(generate_lms_function(norm, mean=mean, p2h=p2h, p97h=p97h),x0).x
    print res
    plt.plot(x, norm.pdf(x, *res), label=mean)
plt.legend()
plt.show()

# # result (distribution)
# # deceased (norm)
# np.array([16.0999999, 1.67365310]) /1000000
# np.array([6.94999864, 1.27959154]) /100000
# np.array([3.08999842, 0.87240868]) /10000
# np.array([8.44000088, 2.22481098]) /10000
# np.array([1.61006021, 0.43275679]) /1000
# np.array([5.95000235, 1.28064016]) /1000
# np.array([9.64999838, 2.09200771]) /500
# np.array([4.28000624, 0.93405922]) /100
# np.array([7.80032793, 2.13982958]) /100
# # severe (norm)
# np.array([16.09999999, 1.67365310]) /1000000
# np.array([4.079997660, 0.84185997]) /10000
# np.array([10.39999943, 2.13262174]) /1000
# np.array([3.429999990, 0.21143857]) /100
# np.array([4.250001620, 0.87760372]) /100
# np.array([8.160003800, 1.68370209]) /100
# np.array([11.79999819, 2.44263192]) /100
# np.array([16.59999901, 3.43586235]) /100
# np.array([18.40000124, 3.77561592]) /100

deceased
[-0.00607273 -0.37988349]
[-0.00542033 -2.66480057]
[ 0.01856955 -5.49975672]
[  409.41038207 -4351.4724888 ]
[0.16171439 0.04365972]
[  5.83023483 -66.28315334]
[ -129.31374767 -1849.90160589]
[4.28000624 0.93405922]
[7.80032793 2.13982958]
severe
[1.12812698e-06 2.00036799e+00]
[ 0.0303215  -3.65811359]
[ -167.22500757 -1684.3821285 ]
[3.42999999 0.21143857]
[4.25000162 0.87760372]
[8.1600038  1.68370209]
[11.79999819  2.44263192]
[16.59999901  3.43586235]
[18.40000124  3.77561592]


/home/thinkstation-ubuntu/.local/lib/python2.7/site-packages/matplotlib/backends/backend_gtk3.py:215: Warning: Source ID 23336 was not found when attempting to remove it
  GLib.source_remove(self._idle_event_id)
